# CREDIT SCORING

## 1. Загрузка библиотек и просмотр данных

In [ ]:
from pandas import Series
import pandas as pd
import numpy as np
import pandas_profiling as pp

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def get_num_info(col, title=None):
    '''Function is called to plot feture distribution'''

    title = title if title is not None else f"Distribution for '{col}"
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5),)
    
    sns.distplot(col.values, bins=20, ax=ax1)
    
    fig = sm.qqplot(col, fit=True, line='45', ax=ax2)
    fig.suptitle(title, fontsize=20)
    
    ax3.boxplot(col.values,vert = False)

    ax1.set_title('QQ-plot')
    ax2.set_title('Distribution')
    ax3.set_title('Boxplot')

    plt.show()

In [ ]:
def detect_outliers(data):
    '''Function is called to detect outliers'''
    q1, q3 = np.percentile(sorted(data), [25, 75])

    IQR = q3 - q1

    l_b = q1 - (1.5 * IQR) # lower bound
    u_b = q3 + (1.5 * IQR) # upper bound
    outl_count = len(data[data < l_b]) + len(data[data > u_b])

    print(
        f'Lower Bound: {round(l_b,3)}, Upper Bound {round(u_b,3)}, Outliers Count: {outl_count}')

In [ ]:
def roc_curve_plot(y_test, y_probs):
       
    fpr, tpr, threshold = roc_curve(y_test, y_probs)
    roc_auc = roc_auc_score(y_test, y_probs)

    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label='Regression')
    plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
def confusion_matrix_plot(y_test, y_pred, xlabels = ['True(P)', 'False(P)'], ylabels = ['True(A)', 'False(A)'], title='', cmap = None):
    cf = confusion_matrix(y_test, y_pred)
    sns.heatmap(cf, annot=True, annot_kws={"size": 20}, fmt='', cbar = False, xticklabels = xlabels, yticklabels= ylabels, cmap = cmap)
    plt.title('Матрица ошибок ' + title)
    plt.show()

In [ ]:
def get_metrics(y_test, y_pred):
    print('Accuracy = %0.4f' % accuracy_score(y_test,y_pred) 
          + '\nPrecision = %0.4f' % precision_score(y_test,y_pred)
          + '\nReacall = %0.4f' % recall_score(y_test,y_pred)
          + '\nF1_score = %0.4f' % f1_score(y_test,y_pred))

In [ ]:
RANDOM_SEED = 42

In [ ]:
data_directory = '/kaggle/input/sf-dst-scoring/'

In [ ]:
df_train = pd.read_csv(data_directory + 'train.csv')
df_test = pd.read_csv(data_directory + 'test.csv')
df_sample = pd.read_csv(data_directory + 'sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_sample.info()

In [ ]:
# объединим тестовую и тренеровочкую выборку в один датасет
df_train['sample'] = 1 # 1 обозначим значения тренировочной выборки
df_test['sample'] = 0 # 0 обозначим значения тестовой выборки
df_test['default'] = -1 # значение, которое необходимо предсказать
df = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
df.sample(5)

**Описание данных:**

client_id - идентификатор клиента

education - уровень образования

sex - пол заемщика

age - возраст заемщика

car - флаг наличия автомобиля

car_type - флаг автомобиля иномарки

decline_app_cnt - количество отказанных прошлых заявок

good_work - флаг наличия “хорошей” работы

bki_request_cnt - количество запросов в БКИ

home_address - категоризатор домашнего адреса

work_address - категоризатор рабочего адреса

income - доход заемщика

foreign_passport - наличие загранпаспорта

sna - связь заемщика с клиентами банка

first_time - давность наличия информации о заемщике

score_bki - скоринговый балл по данным из БКИ

region_rating - рейтинг региона

app_date - дата подачи заявки

default - флаг дефолта по кредиту

## 2. Первичный анализ данных

In [ ]:
df_profile = pp.ProfileReport(df)
df_profile.to_file('result.html')

**[Profile Report - Link](./result.html)**

**Итак**. 

Датасет содержит:
- 19 различных признаков (за вычетом добавленного признака 'sample') из них:
    - 9 категориальных
    - 7 числовых
    - 3 бинарных
- 110 148 записей о клиентах банка без дубликатов
- 478 пропущенных значений

Рассмотрим каждый признак в отдельности:
- client_id - все значения уникальны, является идентификатором клиента, не несет никакой ценности для анализа.


- app_date - содержит информацию о дате подачи заявки в виде текста. Всего 120 различных значений. Необходима обработка.


- education - категориальный признак. Имеет 5 различных значений среди которых около 50% занимает "SCH". Также содержит все имеющиеся пропуски в датасете. Нужна обработка по заполнению пустых значений.


- sex - категориальный признак, т.к. содержит всего 2 значения имеет смысл переделать в бинарную. Соотношение М/Ж примерно одинаковое, женщин больше на 12%.


- age - числовой признак, распределение имеет тяжелый правый хвост, нужна обработка. Средний возраста клиентов - 39 лет. 


- car - бинарный признак, обработка не требуется. Имеется высокая корреляция с признаком 'car_type', что вполне логично.


- decline_app_cnt - числовой признак, имеет тяжелый правый хвост, нужна обработка. Большинство значений(83%) заполнено нулями, что вполне логично. В основном распределение находится в диападоне от 0 до 6.


- good_work - бинарный признак, обработка не требуется. Большинство клиентов не имеют хорошую работу. Есть подозрение, что данный признак не является таким уж значительным при выдаче кредита.


- score_bki - числовой признак, имеет нормальное распределение, все значения отрицатетльные, 93% значений уникальны.


- bki_request_cnt - числовой признак, на графике имеет тяжелый правый хвост, нужна обработка. Много нулей(23%), что нормально. В основном распределение находится в диапазоне от 0 до 8.


- region_rating - числовой признак. Имеет распределение от 20 до 80. Больше похож на категориальный признак


- home_address, work_address - категориальные признаки, имеют 3 значения, обработка не нужна.


- income - числовой признак, на графике имеет тяжелый правый хвост с большим разбросом значений, нужна обработка. 


- sna, first_time категориальные признаки, имеют 4 значения, обработка не нужна.


- foreign_passport - бинарный признак. Большинство клиентов(85%) не имеют загранпаспорта.


- default - бинарная, целевая переменная, видно, что клиентов у которых случился дефолт гораздо меньше.  

В данных о клиентах присутствуют переменные всех типов, не все имеют числовую запись, а также есть пропущенные значения. Значит, нам необходима предварительная очистка данных.

Как мы видим, пропуски встречаются только в информации об образовании клиентов. Посмотрим на признак ближе:

In [ ]:
df['education'].value_counts().plot.barh()

Заполним пропуски в поле 'education' самым частоповторяемым значением 'SCH':

In [ ]:
df.education.fillna('SCH', inplace = True)

## 3. Анализ данных

Полученные признаки сгруппируем в четыре категории по типу их обработки (временные, категориальные, бинарные и числовые) для дальнейшего удобного анализа.

In [ ]:
time_cols = ['app_date']
cat_cols = ['education',  'home_address', 'work_address', 'sna', 'first_time']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
num_cols = ['age','decline_app_cnt','region_rating','score_bki','bki_request_cnt','income']

### Время

Переведем дату подачи заявки в формат datetime

In [ ]:
df['app_date'] = pd.to_datetime(df['app_date'], format='%d%b%Y')

In [ ]:
# определяем начальую дату (1 января 2014 года)
df_min = min(df['app_date'])

In [ ]:
# добавим новый признак - количество прошедших дней с 1 января в году
df['days'] = (df['app_date'] - df_min).dt.days.astype('int') 

In [ ]:
# добавим категориальный признак - месяц, т.к. анализ показал, что участвует всего 4 месяца в одном и том же году
df['month'] = df['app_date'].dt.month.astype('int')

In [ ]:
df['month'].value_counts()

Добавим новые признаки к переменным

In [ ]:
num_cols = num_cols + ['days']
cat_cols = cat_cols + ['month']

Уберем изначальный признак

In [ ]:
df.drop(['app_date'], axis = 1, inplace = True)

In [ ]:
df.sample(5)

### Числовые переменные

#### Графики

In [ ]:
for col in num_cols:
    get_num_info(df[col], title=col)
    detect_outliers(df[col])

#### Логарифмирование

После построения гистограмм стало очевидно, что распределения числовых переменных:
- age
- decline_app_cnt
- bki_request_cnt
- income

имеют тяжёлый правый хвост. 

Сделаем распределение данных переменных более нормальным, логарифмировав величины этих переменных:

In [ ]:
for col in ['age','decline_app_cnt','bki_request_cnt','income']:
    df[col] = (df[col] + 1).transform(np.log)

Построим графики распределения логарифмированных переменных:

In [ ]:
for col in ['age','decline_app_cnt','bki_request_cnt','income']:
    get_num_info(df[col], title=col)
    detect_outliers(df[col])

#### Выбросы

Согласно построенным графикам выбросы имеются в 5-ти переменных:
- decline_app_cnt - т.к. основным значением является 0, то в "выбросы" попало 18677 значений, чем они не являются, оставляем.
- region_rating - 17917 значений находится за границами, не являются выбросами, т.к. находятся в допустимом диапазоне от 0 до 100, оставляем так как есть.
- score_bki - 518 значений находятся за границами, их немного, оставляем.
- bki_request_cnt - 15 значений находятся за границей, их не много, оставляем.
- income - сильный разброс по доходам даже после логорифмирования, есть выбросы как слевой так и справой стороны. Пока удалять не будем

#### Оценка корреляций

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (10,5))
ax = sns.heatmap(df[num_cols + ['default']].corr().abs(), vmin=0, vmax=1, annot = True, fmt='.1g', cmap ='ocean')

Все признаки низкокоррелируемы и пригодны для дальнейшего использования при построении модели

#### Оценка значимости 

Для оценки значимости числовых переменных будем использовать функцию f_classif из библиотеки sklearn. 

Возможности модуля sklearn.feature_selection могут быть использованы не только для выбора важных признаков, но и для уменьшения размерности, улучшения предсказательной силы моделей, либо для повышения их производительности на очень многомерных наборах данных.

В основе метода оценки значимости переменных лежит однофакторный дисперсионный анализ (ANOVA). Основу процедуры составляет обобщение результатов двух выборочных t-тестов для независимых выборок (2-sample t). 

В качестве меры значимости мы будем использовать значение f-статистики. Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для нашей линейной модели.

In [ ]:
imp_num = pd.Series(f_classif(df[num_cols], df['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

**Вывод**

Самой значимой числовой переменной по результатам оценки является - оценка плательщика БКИ (score_bki).
Наименее значимой переменной является возраст (age)

### Категориальные переменные

#### Преобразование переменных в числа

SEX

In [ ]:
label_encoder = LabelEncoder()

mapped_sex = pd.Series(label_encoder.fit_transform(df['sex']))
print(dict(enumerate(label_encoder.classes_)))

EDUCATION

In [ ]:
df.education = pd.Series(label_encoder.fit_transform(df['education']))
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
# Для бинарных признаков мы будем использовать LabelEncoder

label_encoder = LabelEncoder()

for column in bin_cols:
    df[column] = label_encoder.fit_transform(df[column])
    
# убедимся в преобразовании    
df.head()

#### Оценка значимости 

In [ ]:
imp_cat = Series(mutual_info_classif(df[bin_cols + cat_cols], df['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

**Вывод**

Наиболее важным категориальным признаком является - связь заемщика с клиентами банка (sna).

Наименее значимым - пол заемщика (sex).

## 4. Построение и оценка модели

### Модель 1. Первичная модель

#### Построение модели

Подготавливаем данные для обучения

In [ ]:
train = df[df['sample'] == 1]
test = df[df['sample'] == 0]
train = train.drop(['sample', 'client_id'], axis=1)
test = test.drop(['sample', 'client_id', 'default'], axis=1)

In [ ]:
# Задаем зависимую и независимые переменные:

X = train.drop(['default'], axis =1).values
Y = train['default'].values

Разбиваем выборку на обучающую и тестовую и обучаем нашу модель:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_probs = model.predict_proba(X_test)[:,1]

#### Оценка качества модели

In [ ]:
roc_curve_plot(y_test, y_probs)
confusion_matrix_plot(y_test, y_pred, xlabels =['Не дефолт(P)', 'Дефолт(P)'], ylabels =['Не дефолт(P)', 'Дефолт(P)'], title = 'default')
get_metrics(y_test, y_pred)

**Вывод**

Целевая метрика ROC-AUC достаточно высокая - 0.744
Тем не менее из матрицы ошибок видно, что мы практически не угадываем дефолтных клиентов.
В первую очередь это связано с несбалансированностью значений в целевой переменной, которая была отмечена еще при первичном анализе данных. Соответственно precision, recall, F1-score корректнее описывают алгоритм, чем accuracy.
Низкое значение метрик Reacall и F1 говорит о том, что получившася модель - плохая

### Модель 2. Нормализация и oversampling

#### Построение модели

Попробуем исправить ситуацию с несбалансированностью данных применив oversampling.

Увеличим количество записей клиентов, у которых случился дефолт

In [ ]:
num_0 = len(df[df['default']==0])
num_1 = len(df[df['default']==1])
print('Соотношение ДО: \n 0 = {}\n 1 = {}'.format(num_0,num_1))

oversampled_df = pd.concat([ df[df['default']==0] , df[df['default']==1].sample(num_0, replace=True) ])
print('\nCoотношение ПОСЛЕ:')
oversampled_df['default'].value_counts()

Снова подготавливаем наши данные

In [ ]:
train = oversampled_df[oversampled_df['sample'] == 1]
test = oversampled_df[oversampled_df['sample'] == 0]
train = train.drop(['sample', 'client_id'], axis=1)
test = test.drop(['sample', 'client_id', 'default'], axis=1)

В этот раз также проведем нормализацию числовых переменных

In [ ]:
X_num = StandardScaler().fit_transform(train[num_cols].values)

In [ ]:
# Задаем зависимые и независимые переменные:
X = np.hstack([X_num, train[bin_cols+cat_cols].values])
Y = train['default'].values

In [ ]:
# Разбиваем выборку и обучаем подель
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=RANDOM_SEED)
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_probs = model.predict_proba(X_test)[:,1]

#### Оценка качества модели

In [ ]:
roc_curve_plot(y_test, y_probs)
confusion_matrix_plot(y_test, y_pred, xlabels =['Не дефолт(P)', 'Дефолт(P)'], ylabels =['Не дефолт(P)', 'Дефолт(P)'], title = 'default')
get_metrics(y_test, y_pred)

**Вывод**

Совсем другое дело, хоть и точность предсказаний с целевой метрикой ROC-AUC снизились, сама модель стала более стабильной. Метрики Reacall и F1 Score выросли до приличных значений. 

### Модель 3. Подбор гиперпараметров

#### Построение модели

Попробуем улучшить нашу модель подобрав оптимальные для построения гиперпараметры

In [ ]:
# запускаем GridSearch на небольшом кол-ве итераций max_iter=100
model = LogisticRegression(random_state=RANDOM_SEED)

n = 100

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[n]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[n]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[n]},
]

clf = GridSearchCV(model, param_grid, cv=5,  n_jobs=-1, verbose=0)
best_model = clf.fit(X_train, y_train)
model = clf.best_estimator_
##печатаем параметры
best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))
    ##печатаем метрики
y_pred = model.predict(X_test)
y_probs = model.predict_proba(X_test)[:,1]
get_metrics(y_test, y_pred)

In [ ]:
best_model = LogisticRegression( C= 1.0,
                            class_weight= 'none',
                            dual= False,
                            fit_intercept= True,
                            intercept_scaling= 1,
                            l1_ratio= None,
                            max_iter= 100,
                            multi_class= 'auto',
                            n_jobs= None,
                            penalty= 'l2',
                            random_state= 42,
                            solver= 'newton-cg',
                            tol= 0.0001,
                            verbose= 0,
                            warm_start= False)

best_model.fit(X_train, y_train)

y_pred_prob = best_model.predict_proba(X_test)[:,1]
y_pred = best_model.predict(X_test)

#### Оценка качества модели

In [ ]:
roc_curve_plot(y_test, y_probs)
confusion_matrix_plot(y_test, y_pred, ['Не дефолт(P)', 'Дефолт(P)'], ylabels =['Не дефолт(P)', 'Дефолт(P)'], title = 'default')
get_metrics(y_test, y_pred)

**Вывод**

Построение модели на оптимальных параметрах не дало никакого изменения относительно предыдущей версии. Для улучшения результата и точности модели (пока она правильно предсказывает в 65% случаев) стоит сгенерировать больше новых признаков и поработать с выбросами (не хватило времени). Также в работе не были испробованы другие методы машинного обучения, что также могло бы привести к более лучшему результату.
Будем сабмититься

## Submission

In [ ]:
test = df[df['sample'] == 0]
X_num = StandardScaler().fit_transform(test[num_cols].values)
X_test = np.hstack([X_num, test[bin_cols+cat_cols].values])
y_prob = best_model.predict_proba(X_test)[:,1]
test['default'] = y_prob

In [ ]:
submission = test[['client_id','default']]
submission.to_csv('submission.csv', index=False)
submission.sample(3)
#submission.shape